In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from tqdm import tqdm, tqdm_notebook
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import gensim
from gensim import corpora, models
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from prettytable import PrettyTable
import scipy.sparse
import scipy
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
print('CWD is ', os.getcwd())

# Vis Libs..

# Text Libs..

# ML Libs...

CWD is  d:\NEU\Năm 3\SEMESTER 6\ĐỀ ÁN DSEB\Mercari Price Prediction\3. Modelling


# 6. Data Merge

**Getting train data**

In [51]:
file = open('../2. Processing/storage/train_bow_name', 'rb')
train_bow_name = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_bow_desc', 'rb')
train_bow_desc = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_brand', 'rb')
train_brand = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_category', 'rb')
train_category = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_dummies', 'rb')
train_dummies = pickle.load(file)
file.close()

# file = open('../2. Processing/storage/y_train', 'rb')
# y_train = pickle.load(file)
# file.close()

In [52]:
file = open('../2. Processing/storage/df_train_model', 'rb')
df_train_model = pickle.load(file)
file.close()

y_train = np.log1p(df_train_model['price'])

**Getting test data**

In [53]:
file = open('../2. Processing/storage/test_bow_name', 'rb')
test_bow_name = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_bow_desc', 'rb')
test_bow_desc = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_brand', 'rb')
test_brand = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_category', 'rb')
test_category = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_dummies', 'rb')
test_dummies = pickle.load(file)
file.close()

In [54]:
file = open('../2. Processing/storage/df_test_model', 'rb')
df_test_model = pickle.load(file)
file.close()


y_test = np.log1p(df_test_model['price'])

In [55]:
df_test_model

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
401704,401704,Vs pink top,3,Women/Athletic Apparel/Shirts & Tops,PINK,14.0,0,Football season is back! Victoria Secret sweat...
1112208,1112208,iPhone 6 Plus LuMee LED Case BRAND New,1,"Electronics/Cell Phones & Accessories/Cases, C...",Apple,15.0,1,Free Same Day Shipping Brand new in the box Ch...
1136901,1136901,The North Face black M ladies jacket,3,Women/Coats & Jackets/Fleece Jacket,The North Face,25.0,1,Up for sale is a coat that has been worn a few...
1088881,1088881,Rae Dunn Bowls,2,Home/Home Décor/Home Décor Accents,Rae Dunn,30.0,0,Listing is for all 4. Downsizing my collection...
951911,951911,US Polo Assn Blue Shorts Size 40,3,Men/Shorts/Casual Shorts,US POLO ASSN,22.0,0,No description yet
...,...,...,...,...,...,...,...,...
395283,395283,Ion,2,Beauty/Tools & Accessories/Hair Styling Tools,Sephora,16.0,0,Curling iron
1420170,1420170,Dishonored 2: Limited Edition for PS4,1,Electronics/Video Games & Consoles/Games,Sony,48.0,1,Selling a brand new copy of Dishonored 2: Limi...
963259,963259,Victoria's Secret perfume gift set,1,Beauty/Bath & Body/Bath,Victoria's Secret,26.0,0,"New , never used Victoria's secret eau de parf..."
909695,909695,The Dark Tower #1-7,2,Vintage & Collectibles/Book/Comics,NaN,16.0,0,Complete set Stephen King


In [56]:
print("train_bow_name shape:", train_bow_name.shape)
print("train_bow_desc shape:", train_bow_desc.shape)
print("train_brand shape:", train_brand.shape)
print("train_category shape:", train_category.shape)
print("train_dummies shape:", train_dummies.shape)

train_bow_name shape: (1111223, 31796)
train_bow_desc shape: (1111223, 100000)
train_brand shape: (1111223, 4440)
train_category shape: (1111223, 942)
train_dummies shape: (1111223, 7)


**Concatenate all the features that we generated using scipy.sparse.hstack function and converts the datatype in to 'float32' format so as to feed in ML models**

In [57]:
X_train = scipy.sparse.hstack((train_bow_name, train_bow_desc,
                               train_brand, train_category, train_dummies
                               )).tocsr().astype('float32')

X_test = scipy.sparse.hstack((test_bow_name, test_bow_desc,
                              test_brand, test_category, test_dummies
                              )).tocsr().astype('float32')

print('='*50)
print("X_train Shape is ", X_train.shape)
print("y_train Shape is ", y_train.shape)

print('='*50)
print("X_test Shape is ", X_test.shape)
print("y_test Shape is ", y_test.shape)

X_train Shape is  (1111223, 137185)
y_train Shape is  (1111223,)
X_test Shape is  (370634, 137185)
y_test Shape is  (370634,)


In [64]:
file = open('final_processing/X_train', 'wb')
pickle.dump(X_train, file)
file.close()

file = open('final_processing/y_train', 'wb')
pickle.dump(y_train, file)
file.close()

file = open('final_processing/X_test', 'wb')
pickle.dump(X_test, file)
file.close()

file = open('final_processing/y_test', 'wb')
pickle.dump(y_test, file)
file.close()

# 7. Evaluation Metrics

**Root Mean Squared Logarithm Error (RMLSE):**

$$\epsilon = \sqrt{\frac{1}{n}\sum_{i=1}^n(log(p_i+1) - log(a_i+1)^2)}$$

Where: 
- $\epsilon$ is the RMSLE value (score)
- $\mathbf{n}$ is the total number of observations in the (public/private) data set
- $p_i$ is your prediction of price
- $a_i$ is the acutual sale price for $i$
- $log(x)$ is the natural logarithm of $x$

**This code creates the RMSLE scorer function which would be used as Evalation metric for ML models**

In [92]:
def rmsle_score(y, y_pred):
    y = y.reset_index(drop=True)
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1))** 2.0
               for i in range(len(y_pred))]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [93]:
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle_score(preds, labels), False

In [94]:
# Function for RMSE - (Root mean Square error)
def rmse_score(y, y_pred):
    return math.sqrt(mean_squared_error(y, y_pred))

# 8. Baseline Ridge Model

## 8.1. Using Existing Hyperparameters

In [62]:
ridge_model = Ridge(solver='lsqr', fit_intercept=False)
ridge_model.fit(X_train, y_train)

Ridge(fit_intercept=False, solver='lsqr')

In [151]:
preds = np.expm1(ridge_model.predict(X_test))
print('Train RMSLE = ', rmsle_score(np.expm1(y_train), np.expm1(ridge_model.predict(X_train))))
print('Test RMSLE = ', rmsle_score(np.expm1(y_test), preds))
print('='*30)
print('Train RMSE = ', rmse_score(np.expm1(y_train),np.expm1(ridge_model.predict(X_train))))
print('Test RMSE = ', rmse_score(np.expm1(y_test) , preds))

Train RMSLE =  0.4233182195937269
Test RMSLE =  0.47774871679831415
Train RMSE =  28.051584150176037
Test RMSE =  52.23560406243245


In [145]:
list(preds)

[18.515594,
 16.737232,
 23.481516,
 44.496876,
 10.769324,
 11.008446,
 15.214668,
 17.496307,
 41.394226,
 290.65662,
 39.50887,
 12.435188,
 33.82464,
 8.179109,
 14.580488,
 8.82979,
 10.432589,
 29.30363,
 30.489153,
 29.24448,
 37.606327,
 50.039654,
 23.101519,
 11.420397,
 29.877491,
 10.960783,
 13.470698,
 17.245687,
 13.664671,
 22.811573,
 85.94716,
 8.611716,
 18.2099,
 19.922884,
 13.79501,
 12.791015,
 6.2310967,
 30.506357,
 13.514197,
 23.544313,
 12.039182,
 10.138718,
 11.674381,
 5.6547103,
 15.102776,
 31.773594,
 17.555042,
 15.526257,
 16.430553,
 13.903605,
 34.052036,
 10.741959,
 21.536718,
 43.594402,
 98.5262,
 37.299515,
 7.616762,
 9.405646,
 13.83661,
 30.340883,
 24.20216,
 8.992222,
 9.221043,
 13.935153,
 12.187638,
 23.246153,
 7.932104,
 32.482536,
 25.965527,
 12.800023,
 15.497543,
 22.3675,
 25.412214,
 37.703114,
 12.311401,
 8.002496,
 13.743476,
 20.735975,
 18.049013,
 10.286064,
 9.155855,
 28.528923,
 17.70264,
 22.704535,
 40.129036,
 14.49

In [149]:
y_test_exp = np.expm1(y_test)
y_test_exp.to_list()

AttributeError: 'list' object has no attribute 'shape'

In [150]:
pred_result = pd.DataFrame(data = [y_test_exp.to_list(), list(preds)])
pred_result = pred_result.T
pred_result.columns = ['test_true','test_pred']
pred_result

,test_true,test_pred
0,14.0,18.515594
1,15.0,16.737232
2,25.0,23.481516
3,30.0,44.496876
4,22.0,10.769324
...,...,...
370629,16.0,12.429354
370630,48.0,53.477978
370631,26.0,13.142032
370632,16.0,21.016436
